In [23]:
DEBUG = False

In [24]:
cfg = {
    "num_proc": 2,
    "model_name_or_path": "microsoft/deberta-v3-large",
    "data_dir": "../input/fbck2021",
    "trainingargs": {
        "seed": 42,
    }
}

In [25]:
import re
import pickle
import codecs
import warnings
import logging
from functools import partial
from pathlib import Path
from itertools import chain
from text_unidecode import unidecode
from typing import Any, Optional, Tuple

import pandas as pd
from sklearn.model_selection import KFold
from transformers import AutoTokenizer, set_seed

from datasets import Dataset, load_from_disk

def replace_encoding_with_utf8(error: UnicodeError) -> Tuple[bytes, int]:
    return error.object[error.start : error.end].encode("utf-8"), error.end

def replace_decoding_with_cp1252(error: UnicodeError) -> Tuple[str, int]:
    return error.object[error.start : error.end].decode("cp1252"), error.end

codecs.register_error("replace_encoding_with_utf8", replace_encoding_with_utf8)
codecs.register_error("replace_decoding_with_cp1252", replace_decoding_with_cp1252)

def resolve_encodings_and_normalize(text: str) -> str:
    text = (
        text.encode("raw_unicode_escape")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
        .encode("cp1252", errors="replace_encoding_with_utf8")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
    )
    text = unidecode(text)
    return text

def read_text_files(example, data_dir):
    
    id_ = example["essay_id"]
    
    with open(data_dir / "train" / f"{id_}.txt", "r") as fp:
        example["text"] = resolve_encodings_and_normalize(fp.read())
    
    return example

set_seed(cfg["trainingargs"]["seed"])

warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

In [26]:
data_dir = Path(cfg["data_dir"])

train_df = pd.read_csv('../input/2021_data_for_mlm.csv')

if DEBUG: train_df = train_df[:100]

text_ds = Dataset.from_dict({"essay_id": train_df.essay_id.unique()})

text_ds = text_ds.map(
    partial(read_text_files, data_dir=data_dir),
    num_proc=cfg["num_proc"],
    batched=False,
    desc="Loading text files",
)

text_df = text_ds.to_pandas()

train_df["discourse_text"] = [
    resolve_encodings_and_normalize(x) for x in train_df["discourse_text"]
]

train_df = train_df.merge(text_df, on="essay_id", how="left")
    
disc_types = [
    "Claim",
    "Concluding Statement",
    "Counterclaim",
    "Evidence",
    "Lead",
    "Position",
    "Rebuttal",
]

type2id = {'Lead': 0,
 'Position': 1,
 'Claim': 2,
 'Evidence': 3,
 'Counterclaim': 4,
 'Rebuttal': 5,
 'Concluding Statement': 6,
 'Other': 7}

label2id = {
    "Ineffective": 0,
    "Adequate": 1,
    "Effective": 2,
}

tokenizer = AutoTokenizer.from_pretrained(cfg["model_name_or_path"])

Loading text files #0: 100%|████████████████████████████████████| 7797/7797 [00:08<00:00, 881.89ex/s]

Loading text files #1: 100%|████████████████████████████████████| 7797/7797 [00:09<00:00, 838.10ex/s]


In [27]:
def find_positions(example):

    text = example["text"][0]
    
    # keeps track of what has already
    # been located
    min_idx = 0
    
    # stores start and end indexes of discourse_texts
    idxs = []
    
    for dt in example["discourse_text"]:
        # calling strip is essential
        matches = list(re.finditer(re.escape(dt.strip()), text))
        
        # If there are multiple matches, take the first one
        # that is past the previous discourse texts.
        if len(matches) > 1:
            for m in matches:
                if m.start() >= min_idx:
                    break
        # If no matches are found
        elif len(matches) == 0:
            idxs.append([-1]) # will filter out later
            print('!!!! MISS !!!')
            print(dt.strip())
            print('!!here!!')
            print(text)
            print()
            continue  
        # If one match is found
        else:
            m = matches[0]
            
        idxs.append([m.start(), m.end()])

        min_idx = m.start()

    return idxs

def tokenize(example):
    example["idxs"] = find_positions(example)
    
    # print("New example")
    # print(example["idxs"])
    # print()

    text = example["text"][0]
    text = text.replace('\n', '|')
    chunks = []
    chunk_example = []
    chunk_idxs = []
    examples_classes = [type2id[disc_type] for disc_type in example["discourse_type"]]
    examples_scores = [label2id[disc_effect] for disc_effect in example["discourse_effectiveness"]]
    
    prev = 0

    zipped = zip(
        example["idxs"],
        example["discourse_type"],
        example["discourse_effectiveness"],
    )
    
    for idxs, disc_type, disc_effect in zipped:
        # when the discourse_text wasn't found
        if idxs == [-1]:
            chunk_idxs.append([-1])
            chunk_example.append(-1)
            chunks.append('')
            continue

        s, e = idxs

        # if the start of the current discourse_text is not 
        # at the end of the previous one.
        # (text in between discourse_texts)
        if s != prev:
            chunk_idxs.append([prev,s])
            chunk_example.append(-1)
            chunks.append(text[prev:s])
            prev = s

        # if the start of the current discourse_text is 
        # the same as the end of the previous discourse_text
        if s == prev:
            chunk_idxs.append([s,e])
            chunks.append(text[s:e])
            chunk_example.append(1)
        
        prev = e
        
    input_ids = [tokenizer.cls_token_id]
    token_class_labels = [-100]
    token_scores_labels = [-100]
    token_examples_mapping = [-100]
    
    assert len(examples_classes) == len(examples_scores) 
    assert len(chunks) == len(chunk_idxs) 
    assert len(examples_classes) == len(example["discourse_effectiveness"])

    i = 0
    
    for j, chunk in enumerate(chunks):
        chunk_ids = tokenizer(chunk, padding=False, truncation=False, add_special_tokens=False)
        chunk_ids = chunk_ids['input_ids']
        if len(chunk_ids) == 0: 
            assert chunk_example[j] == -1
            continue
            
        if chunk_example[j] == -1:
            input_ids.extend(chunk_ids)
            token_class_labels += [-100] * len(chunk_ids)
            token_scores_labels += [-100] * len(chunk_ids)
            token_examples_mapping += [-100] * len(chunk_ids)
        if chunk_example[j] == 1: 
            input_ids.extend(chunk_ids)
            token_class_labels += [examples_classes[i]] * len(chunk_ids)
            token_scores_labels += [examples_scores[i]] * len(chunk_ids)
            token_examples_mapping += [i] * len(chunk_ids)

            # DEBUG
            # print(i)
            # print('class', examples_classes[i])
            # print('score', examples_scores[i])
            # ss,ee = example["idxs"][i]
            # print(text[ss:ee])
            # print('***********************')
            # print(tokenizer.decode(chunk_ids))
            # print('***********************')
            # print()            
            # DEBUG
            
            i += 1
            
              
    # print(example["idxs"])
        
    # if (i+1 < len(example["idxs"])):
    #     print('ouch!!!!')
    #     for sss,eee in example["idxs"]:
    #           print(text[sss:eee])
        
    input_ids += [tokenizer.sep_token_id]
    token_class_labels += [-100]
    token_scores_labels += [-100]
    token_examples_mapping += [-100]
    attention_mask = [1] * len(input_ids)

    example['input_ids'] = input_ids
    example['attention_mask'] = attention_mask
    example['token_class_labels'] = token_class_labels
    example['token_scores_labels'] = token_scores_labels
    example['token_examples_mapping'] = token_examples_mapping
    example['examples_scores'] = examples_scores
    example['examples_classes'] = examples_classes
    
    return example

In [28]:

# make lists of discourse_text, discourse_effectiveness
# for each essay
grouped = train_df.groupby(["essay_id"]).agg(list)

ds = Dataset.from_pandas(grouped)

ds = ds.map(
    tokenize,
    batched=False,
    
)   

 97%|███████████████████████████████████████████████████████▍ | 15181/15594 [01:50<00:02, 145.90ex/s]

!!!! MISS !!!
This whole thing is point less how they have us in here for two days im missing my education. We could have finished this in one day and had the rest of the week to get back on the track of learning. I've missed both days of weight lifting, algebra, and my world history that i do not want to fail again! If their are any people actually gonna sit down and take the time to read this then

DO NOT DO THIS NEXT YEAR

.

They are giving us cold lunches. ham and cheese and an apple, I am 16

years old and my body needs proper food. I wouldnt be complaining if they served actual breakfast. but because of Michelle Obama and her healthy diet rule they surve us 1 poptart in the moring. How does the school board expect us to last from 7:05-12:15 on a pop tart? then expect us to get A's, we are more focused on lunch than anything else. I am about done so if you have the time to read this even though this does not count. Bring PROPER_NAME a big Mac from mc donalds, SCHOOL_NAME, (idk ar

100%|█████████████████████████████████████████████████████████| 15594/15594 [01:53<00:00, 137.80ex/s]


In [29]:
ds

Dataset({
    features: ['Unnamed: 0', 'discourse_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'text', 'essay_id', 'idxs', 'input_ids', 'attention_mask', 'token_class_labels', 'token_scores_labels', 'token_examples_mapping', 'examples_scores', 'examples_classes'],
    num_rows: 15594
})

In [30]:
bad_matches = []
for id_, l, ids, dt, tem in zip(ds["essay_id"], ds["examples_scores"], ds["input_ids"], grouped.discourse_text,
                               ds["token_examples_mapping"]):
    
    # count number of labels (ignoring -100)
    num_cls_label = len(set(tem)) - 1
    # count number of cls ids
    num_cls_id = max(tem) + 1
    # true number of discourse_texts
    num_dt = len(dt)
    # print(num_cls_label, num_cls_id, num_dt)
    
    if num_cls_label != num_dt or num_cls_id != num_dt:
        bad_matches.append((id_, l, ids, dt))
        
print("Num bad matches", len(bad_matches))
# temp = train_df[train_df["essay_id"]==bad_matches[0][0]]
# temp_txt = temp.text.values[0]
# print(temp_txt)
# print("*"*100)
# print([x for x in temp.discourse_text if x.strip() not in temp_txt])

Num bad matches 1


In [31]:
for t in ds[0]["discourse_text"]:
    print(t, "\n")
print("*"*100)
print(tokenizer.decode(ds[0]["input_ids"]))
print("*"*100)
print(ds[0]["text"][0])

Some people belive that the so called "face" on mars was created by life on mars. This is not the case. The face on Mars is a naturally occuring land form called a mesa.  

It was not created by aliens, and there is no consiracy to hide alien lifeforms on mars. There is no evidence that NASA has found that even suggests that this face was created by aliens.  

A mesa is a naturally occuring rock formation, that is found on Mars and Earth.  

This "face" on mars only looks like a face because humans tend to see faces wherever we look, humans are obviously extremely social, which is why our brain is designed to recognize faces.  

Many conspiracy theorists believe that NASA is hiding life on Mars from the rest of the world.  

These people would be very wrong. If NASA found life on Mars, then they would get millions of people's attention.  

NASA's budget would increase drasticly, which means that their workers would get paid more. There is no good reason that NASA would hide life on Mar

In [32]:
pdf = ds.to_pandas()
essays = pd.read_csv('../input/feedback-effective-folds/essay_scores.csv')
essays = essays[['essay_id', 'fold']]
essays.head()

,essay_id,fold
0,F98E8D4EA700,0
1,66BB82BD76B2,0
2,85F4C57672EA,0
3,06936C8AA35D,0
4,61C3ADEA1DD5,0


In [33]:
pdfm = pd.merge(pdf, essays, on='essay_id', how='left')

In [34]:
len(pdf), len(pdfm), len(essays)

(15594, 15594, 4191)

In [35]:
all([len(x) > 0 for x in pdfm.token_class_labels.values])

True

In [36]:
pdfm.fold.fillna(-1, inplace=True)
pdfm.fold.value_counts()

-1.0    11403
 0.0      839
 1.0      838
 4.0      838
 2.0      838
 3.0      838
Name: fold, dtype: int64

In [37]:
# pdfm.input_ids.loc[313]

In [38]:
list_cols = ['input_ids',
       'attention_mask', 'token_class_labels', 'token_scores_labels',
       'token_examples_mapping', 'examples_scores', 'examples_classes']

In [39]:
for c in list_cols:
    pdfm[c] = [x.tolist() for x in pdfm[c].values]

In [40]:
import pickle
with open(f'pretraining-{cfg["model_name_or_path"].split("/")[1]}-nbroad.pickle', 'wb') as handle:
    pickle.dump(pdfm, handle, protocol=pickle.HIGHEST_PROTOCOL)